In [2]:
import os 

os.chdir("/home/theovincent/MVA/3DPointCloud/3DPointCloudClassification")

In [15]:
import numpy as np
from classifier_3D.utils.ply_file import read_ply, write_ply
from range_net import Z_GROUNDS, Z_GROUND, MAX_HEIGHT, CITY_LABELS_TO_NUMBERS


cloud, headers = read_ply("data/test/MiniDijon9_with_features_200000_filter_r.ply")
cloud_range_net, headers_range_net = read_ply("data/test/MiniDijon9_with_range_net_1000_samples.ply")

points = np.vstack((cloud["x"], cloud["y"], cloud["z"])).T.astype(np.float32)


# The points that were used with RangeNet++ where the ones respecting z - (avg_z_cloud - avg_z_rangenet) <= max_height
# i.e. z <= H + avg_z_cloud - avg_z_rangenet
used_by_rangenet = points[:, 2] <= MAX_HEIGHT + Z_GROUNDS["MiniDijon9"] - Z_GROUND
predictions = cloud["prediction"]
predictions[used_by_rangenet] = cloud_range_net["class"][used_by_rangenet]

predictions[~used_by_rangenet & (predictions == CITY_LABELS_TO_NUMBERS["Cars"])] = CITY_LABELS_TO_NUMBERS["Building"]
assert (predictions == 0).sum() == 0, "No unclassified points should be output"

write_ply("data/test/MiniDijon9_merged.ply", (points, predictions), ["x", "y", "z", "prediction"])

True